In [2]:
import requests
import time
import pandas as pd
from collections import deque
from bs4 import BeautifulSoup

In [3]:
cambodia_provinces = [
    "Phnom Penh",          # Capital (Autonomous Municipality)
    "Banteay Meanchey",
    "Battambang",
    "Kampong Cham",
    "Kampong Chhnang",
    "Kampong Speu",
    "Kampong Thom",
    "Kampot",
    "Kandal",
    "Kep",
    "Koh Kong",
    "Kratié",
    "Mondulkiri",
    "Oddar Meanchey",
    "Pailin",
    "Preah Vihear",
    "Prey Veng",
    "Pursat",
    "Ratanakiri",
    "Siem Reap",
    "Sihanoukville",
    "Stung Treng",
    "Svay Rieng",
    "Takeo",
    "Tboung Khmum"
]


In [4]:
url_origin = 'https://www.century21global.com/en/l/'
url_origin_info = 'https://www.century21global.com/api/aggregator-service/aggregator/listing-aggregator/'

USER_AGENT  = ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
               "AppleWebKit/537.36 (KHTML, like Gecko) "
               "Chrome/117.0.0.0 Safari/537.36 Edg/117.0.2045.60")

headers = {"User-Agent": USER_AGENT}

In [5]:
result = []
for j in ['land-for-sale', 'homes-for-sale','commercial-for-sale']:
    print(f"\n\n\nScraping {j}...")
    i = 1
    while True:
        print(f"Scraping page {i}...")
        # Send request
        url = url_origin + f'{j}/cambodia?' + f'page={i}&max=10'
        response = requests.get(url, headers=headers)

        # Check request success
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find all property cards
            property_divs = soup.find_all('div', class_='col-md-6 col-xxl-4 ng-star-inserted')

            # Extract and print <a> tag ids
            for div in property_divs:
                a_tag = div.find('a', id=True)
                if a_tag:
                    result.append(a_tag['id'])
                    # print(a_tag['id'])

            if len(property_divs) == 0:
                break
            else:
                i += 1
        else:
            print(f"Request failed with status code: {response.status_code}")







Scraping land-for-sale...
Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Scraping page 30...
Scraping page 31...
Scraping page 32...
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping page 36...
Scraping page 37...
Scraping page 38...
Scraping page 39...



Scraping homes-for-sale...
Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...

In [6]:
print(len(result))
print(result)

1095
['rl-128119873', 'rl-121361083', 'rl-121336928', 'rl-121336927', 'rl-121014027', 'rl-121013870', 'rl-128345072', 'rl-135308223', 'rl-121765074', 'rl-121013853', 'rl-121354381', 'rl-121014000', 'rl-121013857', 'rl-121013848', 'rl-121591502', 'rl-121013837', 'rl-121591503', 'rl-121434164', 'rl-121341607', 'rl-121014026', 'rl-121341608', 'rl-134353287', 'rl-121354391', 'rl-121354390', 'rl-121319946', 'rl-121361089', 'rl-121361088', 'rl-135334522', 'rl-134710761', 'rl-121383044', 'rl-135230279', 'rl-126478627', 'rl-121308582', 'rl-121354382', 'rl-121354383', 'rl-134663860', 'rl-128324443', 'rl-127252060', 'rl-126478628', 'rl-121013877', 'rl-121354388', 'rl-121354387', 'rl-121013809', 'rl-121013873', 'rl-121361095', 'rl-121361094', 'rl-129177636', 'rl-121847474', 'rl-121614969', 'rl-121013977', 'rl-121814479', 'rl-124601398', 'rl-128446369', 'rl-134789643', 'rl-133718943', 'rl-121271676', 'rl-121232432', 'rl-134315478', 'rl-134893615', 'rl-134794010', 'rl-134893648', 'rl-134952270', 'r

In [7]:
all_results = []
for i in range(len(result)):
    print(f"Scraping property {i+1}/{len(result)}")
    temp = {}
    temp_url = url_origin_info + f'{result[i]}/EN?translateDescription=false'
    response = requests.get(temp_url, headers=headers)

    # Check request success
    if response.status_code == 200:
        response = response.json()
        
        temp['id'] = response['id']
        temp['information'] = response['mainRemark']
        temp['type'] = response['basicInfo']['purpose']
        temp['province'] = response['basicInfo']['address']['state']
        temp['price'] = response['basicInfo']['originalPrice']['value'] if response['basicInfo']['originalPrice'] != None else None
        temp['land_size'] = response['basicInfo']['landSize']['value']
        temp['price/m2'] = float(temp['price']) / float(temp['land_size']) if temp['land_size'] != None and temp['price'] != None else None
        temp['lat'] = response['basicInfo']['location']['lat'] if response['basicInfo']['location'] != None else None
        temp['long'] = response['basicInfo']['location']['lon'] if response['basicInfo']['location'] != None else None
        all_results.append(temp)

    else:
        print(f"Request failed with status code: {response.status_code}")

Scraping property 1/1095
Scraping property 2/1095
Scraping property 3/1095
Scraping property 4/1095
Scraping property 5/1095
Scraping property 6/1095
Scraping property 7/1095
Scraping property 8/1095
Scraping property 9/1095
Scraping property 10/1095
Scraping property 11/1095
Scraping property 12/1095
Scraping property 13/1095
Scraping property 14/1095
Scraping property 15/1095
Scraping property 16/1095
Scraping property 17/1095
Scraping property 18/1095
Scraping property 19/1095
Scraping property 20/1095
Scraping property 21/1095
Scraping property 22/1095
Scraping property 23/1095
Scraping property 24/1095
Scraping property 25/1095
Scraping property 26/1095
Scraping property 27/1095
Scraping property 28/1095
Scraping property 29/1095
Scraping property 30/1095
Scraping property 31/1095
Scraping property 32/1095
Scraping property 33/1095
Scraping property 34/1095
Scraping property 35/1095
Scraping property 36/1095
Scraping property 37/1095
Scraping property 38/1095
Scraping property 39/

In [8]:
result[301]

'rl-133884683'

In [9]:
all_results

[{'id': 'rl-128119873',
  'information': '📢 ដី និង ផ្ទះសម្រាប់លក់ \r\n👉  ទីតាំង បឹងកេងកង១ \r\n👉 តម្លៃលក់ $10.000.000\r\n✅ទំហំដី 38m x 30m\r\n✅ ទំហំក្នុងប្លង់ : 1140m² ( ប្លង់រឹង )\r\n✅ទីតាំងល្អ នៅលើផ្លូវ ចរាចរណ៍ ជិតផ្សារសាលារៀន ហើយល្អសម្រាប់បើកអាជីវកម្មគ្រប់ប្រភេទ និង សាងសងជា អគារធំៗ។\r\n✅ លេខអាជ្ញាប័ណ្ណភ្នាក់ងារអចលនវត្ថុ: E-14-160\r\n✅ ពត័មានលំអិតទាក់ទងលេខ 099 666 722 / 099 666 726\r\n#forsale #ហ្វូជីរៀលធី #fujiRealty #PhnomPenh #Odem #realestateagency #propertyforsale #propertyforrent #propertyforlease #flateforsale #houseforsale #condoforsale #condoforren',
  'type': 'LAND',
  'province': 'Phnom Penh',
  'price': 10000000.0,
  'land_size': 1140.0,
  'price/m2': 8771.929824561403,
  'lat': 11.546709,
  'long': 104.917275},
 {'id': 'rl-121361083',
  'information': 'ដីលក់ នៅសង្កាត់ជ្រាវ ជាប់ផ្លូវក្រវ៉ាត់ក្រុងទី2 សៀមរាប\r\n តមៃ្ល $ 180/m² (ចរចា)\r\n ខាងមុខ\u200bជាប់ផ្លូវធំ ខាងក្រោយជាប់ផ្លូវលំ អាចពុះឡូត៍ សង់សណ្ឋាគារ ភោជនីយដ្ឋាន  រឺឃ្លាំងដាក់ ទំនិញផ្សេងៗបាន\r\n ផ្ទៃ\u200bក្រលា 5072m² (ទំហ

In [10]:
df_result_all = pd.DataFrame(all_results)
df_result_all

,id,information,type,province,price,land_size,price/m2,lat,long
0,rl-128119873,📢 ដី និង ផ្ទះសម្រាប់លក់ \r\n👉 ទីតាំង បឹងកេងកង...,LAND,Phnom Penh,10000000.0,1140.0,8771.929825,11.546709,104.917275
1,rl-121361083,ដីលក់ នៅសង្កាត់ជ្រាវ ជាប់ផ្លូវក្រវ៉ាត់ក្រុងទី2...,LAND,Siemreap,9130000.0,5362.4,1702.595853,NaN,NaN
2,rl-121336928,ដីកែងលក់លេីផ្លូវកែងបួនជ្រុង (ផ្លូវ ៥៩៨ ឬ ផ្លូវ...,LAND,Phnom Penh,6400000.0,5434.0,1177.769599,11.636381,104.880046
3,rl-121336927,ដីកែងលក់លេីផ្លូវកែងបួនជ្រុង (ផ្លូវ ៥៩៨ ឬ ផ្លូវ...,LAND,Phnom Penh,6400000.0,5334.0,1199.850019,11.636381,104.880046
4,rl-121014027,Please contact me at: 012389108 / 098 389 108\...,LAND,Phnom Penh,6000000.0,5940.0,1010.101010,11.635246,104.881119
...,...,...,...,...,...,...,...,...,...
1090,rl-121591495,ឃ្លាំងសម្រាប់លក់នៅផ្លូវហាន់ណូយ\r\nនៅជាប់ផ្លូវហ...,COMMERCIAL,Phnom Penh,1300.0,1870.0,0.695187,11.624153,104.857137
1091,rl-121707765,"ឃ្លាំងសម្រាប់លក់នៅទួលសង្កែរ\r\n* នៅជិតផ្សារ, ទ...",COMMERCIAL,Phnom Penh,1300.0,499.0,2.605210,11.590668,104.909745
1092,rl-121577831,🏠 ផ្ទះអាជីវកម្មសម្រាប់ជួលនៅបុរីប៉េងហួត\r\n* ទី...,COMMERCIAL,Phnom Penh,600.0,63.0,9.523810,11.524933,104.956408
1093,rl-121577830,🏠 ផ្ទះអាជីវកម្មសម្រាប់ជួលនៅបុរីប៉េងហួត បឹងស្នោ...,COMMERCIAL,Phnom Penh,500.0,92.4,5.411255,11.533086,104.957604


In [11]:
df_result_all.to_csv('../../data/raw/century21.csv', index=False)

In [12]:
import pandas as pd
df = pd.read_csv('../../data/raw/century21.csv')
df

,id,information,type,province,price,land_size,price/m2,lat,long
0,rl-128119873,📢 ដី និង ផ្ទះសម្រាប់លក់ \r\n👉 ទីតាំង បឹងកេងកង...,LAND,Phnom Penh,10000000.0,1140.0,8771.929825,11.546709,104.917275
1,rl-121361083,ដីលក់ នៅសង្កាត់ជ្រាវ ជាប់ផ្លូវក្រវ៉ាត់ក្រុងទី2...,LAND,Siemreap,9130000.0,5362.4,1702.595853,NaN,NaN
2,rl-121336928,ដីកែងលក់លេីផ្លូវកែងបួនជ្រុង (ផ្លូវ ៥៩៨ ឬ ផ្លូវ...,LAND,Phnom Penh,6400000.0,5434.0,1177.769599,11.636381,104.880046
3,rl-121336927,ដីកែងលក់លេីផ្លូវកែងបួនជ្រុង (ផ្លូវ ៥៩៨ ឬ ផ្លូវ...,LAND,Phnom Penh,6400000.0,5334.0,1199.850019,11.636381,104.880046
4,rl-121014027,Please contact me at: 012389108 / 098 389 108\...,LAND,Phnom Penh,6000000.0,5940.0,1010.101010,11.635246,104.881119
...,...,...,...,...,...,...,...,...,...
1090,rl-121591495,ឃ្លាំងសម្រាប់លក់នៅផ្លូវហាន់ណូយ\r\nនៅជាប់ផ្លូវហ...,COMMERCIAL,Phnom Penh,1300.0,1870.0,0.695187,11.624153,104.857137
1091,rl-121707765,"ឃ្លាំងសម្រាប់លក់នៅទួលសង្កែរ\r\n* នៅជិតផ្សារ, ទ...",COMMERCIAL,Phnom Penh,1300.0,499.0,2.605210,11.590668,104.909745
1092,rl-121577831,🏠 ផ្ទះអាជីវកម្មសម្រាប់ជួលនៅបុរីប៉េងហួត\r\n* ទី...,COMMERCIAL,Phnom Penh,600.0,63.0,9.523810,11.524933,104.956408
1093,rl-121577830,🏠 ផ្ទះអាជីវកម្មសម្រាប់ជួលនៅបុរីប៉េងហួត បឹងស្នោ...,COMMERCIAL,Phnom Penh,500.0,92.4,5.411255,11.533086,104.957604


In [13]:
df = df[~df['lat'].isna()].reset_index()
df.drop(columns=['index'], inplace=True)
df


,id,information,type,province,price,land_size,price/m2,lat,long
0,rl-128119873,📢 ដី និង ផ្ទះសម្រាប់លក់ \r\n👉 ទីតាំង បឹងកេងកង...,LAND,Phnom Penh,10000000.0,1140.0,8771.929825,11.546709,104.917275
1,rl-121336928,ដីកែងលក់លេីផ្លូវកែងបួនជ្រុង (ផ្លូវ ៥៩៨ ឬ ផ្លូវ...,LAND,Phnom Penh,6400000.0,5434.0,1177.769599,11.636381,104.880046
2,rl-121336927,ដីកែងលក់លេីផ្លូវកែងបួនជ្រុង (ផ្លូវ ៥៩៨ ឬ ផ្លូវ...,LAND,Phnom Penh,6400000.0,5334.0,1199.850019,11.636381,104.880046
3,rl-121014027,Please contact me at: 012389108 / 098 389 108\...,LAND,Phnom Penh,6000000.0,5940.0,1010.101010,11.635246,104.881119
4,rl-121013870,ដីលក់បន្ទាន់ នៅស្រុកស្អាង ខេត្តកណ្តាល [English...,LAND,Kandal,5969900.0,59699.0,100.000000,11.264608,104.973933
...,...,...,...,...,...,...,...,...,...
1059,rl-121591495,ឃ្លាំងសម្រាប់លក់នៅផ្លូវហាន់ណូយ\r\nនៅជាប់ផ្លូវហ...,COMMERCIAL,Phnom Penh,1300.0,1870.0,0.695187,11.624153,104.857137
1060,rl-121707765,"ឃ្លាំងសម្រាប់លក់នៅទួលសង្កែរ\r\n* នៅជិតផ្សារ, ទ...",COMMERCIAL,Phnom Penh,1300.0,499.0,2.605210,11.590668,104.909745
1061,rl-121577831,🏠 ផ្ទះអាជីវកម្មសម្រាប់ជួលនៅបុរីប៉េងហួត\r\n* ទី...,COMMERCIAL,Phnom Penh,600.0,63.0,9.523810,11.524933,104.956408
1062,rl-121577830,🏠 ផ្ទះអាជីវកម្មសម្រាប់ជួលនៅបុរីប៉េងហួត បឹងស្នោ...,COMMERCIAL,Phnom Penh,500.0,92.4,5.411255,11.533086,104.957604


In [14]:
from deep_translator import GoogleTranslator

def safe_translate(text):
    if isinstance(text, str) and text.strip():
        return GoogleTranslator(source='km', target='en').translate(text)
    return None

df['translate'] = df['information'].apply(safe_translate)



In [15]:
df[df['price'].isnull()]

,id,information,type,province,price,land_size,price/m2,lat,long,translate
295,rl-133884683,ដីលក់បន្ទាន់⚡️នៅផ្លូវជាតិលេខ21ក្បែរផ្លូវក្រវ៉ា...,LAND,Kandal,NaN,1843.0,NaN,11.360443,105.005444,Land Emergency Land ⚡️ N by National Road 21 N...
296,rl-121013970,ID: CKOSIJJG\r\n🏠ដី សម្រាប់លក់ នៅសែនសុខ!\r\nដី...,LAND,Phnom Penh,NaN,3000.0,NaN,11.579789,104.857097,ID: ckosijjg\r\n🏠 Land for sale at Sen Sok\r\n...
297,rl-121013838,ដីនិងផ្ទះលក់ នៅទួលគោក ស្ថិតក្នុងតំបន់មានសក្តាន...,LAND,Phnom Penh,NaN,600.0,NaN,11.571095,104.888157,Land and home in Toul Kork is potentially in t...
298,rl-121591501,ដីសម្រាប់លក់នៅខណ្ឌដង្កោរ\r\n* នៅជិតតំបន់សេដ្ឋក...,LAND,Phnom Penh,NaN,NaN,NaN,11.439065,104.854218,Land for sale in Khan Dangkor\r\n* Near the Sp...
299,rl-121789127,ដីសម្រាប់លក់នៅសង្កាត់ភ្នំពេញថ្មី\r\n* នៅជិតផ្ស...,LAND,Phnom Penh,NaN,600.0,NaN,11.573510,104.877057,Land for sale in Sangkat Phnom Penh Thmey\r\n*...
...,...,...,...,...,...,...,...,...,...,...
368,rl-135329062,ដីមានបន្ទប់ជួល លក់បន្ទាន់⚡️នៅក្រោយផ្សារបឹងបៃតង...,LAND,Phnom Penh,NaN,640.0,NaN,11.624212,104.857043,Land has an instant rental\r\n👉 Price: $ 950 /...
369,rl-135327262,ដីកែងខ្នាតវីឡាលក់បន្ទាន់⚡️នៅជិតផ្លូវជាតិលេខ21 ...,LAND,Kandal,NaN,1800.0,NaN,11.360443,105.005444,Porn Land Seal Emergency Sale ⚡️ Near National...
1002,rl-133983503,ដីនិងផ្ទះលក់⚡️នៅជិតផ្លូវជាតិលេខ២ ក្រុងតាខ្មៅ (...,RESIDENTIAL,Kandal,NaN,1800.0,NaN,11.481382,104.944869,Land & Home Sales ⚡️ Near National Road No 2 T...
1003,rl-132369484,8 Bedroom Villa for Sale Area Toul Kork\r\n- S...,RESIDENTIAL,Phnom Penh,NaN,717.0,NaN,11.576372,104.893521,8 BedRoom Villa for Sale Area Toul Kork\r\n- S...


In [16]:
df.to_csv('../../data/raw/century21_temp.csv', index=False)

In [ ]:
import os
print(os.listdir("./Llama-3.1-8B-Instruct"))


Checking if model exists at ./Llama-3.1-8B-Instruct...
Model not found at ./Llama-3.1-8B-Instruct. Downloading meta-llama/Llama-3.1-8B-Instruct...


Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]


Error downloading model: 401 Client Error. (Request ID: Root=1-68390f05-31eb39ea63783e8d44a380e4;38426eb3-fbd4-431b-b944-f5c7c90184e2)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct/resolve/0e9e39f249a16976918f6564b8830bc894c89659/.gitattributes.
Access to model meta-llama/Llama-3.1-8B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.
Please ensure you have requested and been granted access to the model on Hugging Face.
You might also need to be logged into Hugging Face CLI: `huggingface-cli login`


In [ ]:
from transformers import pipeline
import torch

# Use GPU if available
device = 0 if torch.cuda.is_available() else -1

# Load Llama-3 for text generation (acts like QA)
qa_pipeline = pipeline(
    "text-generation",
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    torch_dtype=torch.float16,
    device_map="auto" if device == 0 else None
)

# Define your input in chat format (Llama-3 is chat-style)
context = "The capital city of Cambodia is Phnom Penh. It is known for its rich history and culture."
question = "What is the capital city of Cambodia?"

# Format input for Llama-3 (chat-style prompt)
prompt = f"<|system|>You are a helpful assistant.<|user|>Context: {context}\nQuestion: {question}<|assistant|>"

# Run the pipeline
response = qa_pipeline(prompt, max_new_tokens=50, do_sample=False)[0]['generated_text']

# Extract only the answer from the output (after "<|assistant|>")
answer = response.split("<|assistant|>")[-1].strip()

print("Answer:", answer)


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct.
403 Client Error. (Request ID: Root=1-68391077-2208df076582531b19f9eb0d;9e5db8b5-4f8a-48dd-9335-8f16e2e03cb6)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B-Instruct is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct to ask for access.

In [24]:
dfna = df[df['price/m2'].isna()].reset_index(drop=True)
dfna

,id,information,type,province,price,land_size,price/m2,lat,long,translate
0,rl-121013794,[Land for sale-English below]\r\nដីសម្រាប់លក់!...,LAND,Phnom Penh,500000.0,NaN,NaN,11.660139,104.909685,[Land for Sale-English Below]\r\nLand for sale...
1,rl-121367458,"ផ្ទះលក់\r\n- តម្លៃ : $290,000(ចចារ)(P-3317)\r\...",LAND,Phnom Penh,290000.0,NaN,NaN,11.545763,104.930554,"House for sale\r\n- Price: $ 290,000 (p-3317)\..."
2,rl-121434153,ផ្ទះសម្រាប់លក់​ នៅបុរីមហាសែនសុខ\r\n- តំលៃ : $9...,LAND,Phnom Penh,98000.0,NaN,NaN,11.573510,104.877057,House for sale in Borey Great Sopheak\r\n- Pri...
3,rl-121424857,"ដីលក់នៅសួនទឹកកាំងម៉េង(L-3349)\r\n- តម្លៃ : 85,...",LAND,Phnom Penh,85000.0,NaN,NaN,11.450613,104.898006,Land for sale at the Park Meng (3349)\r\n- Pri...
4,rl-121424852,"ដីលក់ តម្លៃទាបជាងទីផ្សារ(L-3354)\r\nតម្លៃ: $2,...",LAND,Phnom Penh,2900.0,NaN,NaN,11.589422,104.927091,Land for sale lower market (L-3354)\r\nPrice: ...
...,...,...,...,...,...,...,...,...,...,...
242,rl-132369484,8 Bedroom Villa for Sale Area Toul Kork\r\n- S...,RESIDENTIAL,Phnom Penh,NaN,717.0,NaN,11.576372,104.893521,8 BedRoom Villa for Sale Area Toul Kork\r\n- S...
243,rl-128240289,📢 ផ្ទះអាជីវកម្ម សម្រាប់ លក់ ទីតំាង បឹងគេងកងទី១...,RESIDENTIAL,Phnom Penh,NaN,216.0,NaN,11.546709,104.917275,📢 Business home for sale Lake location 1\r\n👉 ...
244,rl-121014059,Please contact me at: 010898864\r\nផ្ទះលក់ (ល្...,COMMERCIAL,Phnom Penh,580000.0,NaN,NaN,11.579752,104.912604,PLEASE Contact ME AT: 010898864\r\nHouse for s...
245,rl-121434155,ផ្ទះអាជីវកម្មសម្រាប់លក់ នៅជិតបុរីឈូកវ៉ា1(L-336...,COMMERCIAL,Phnom Penh,168000.0,NaN,NaN,11.573510,104.877057,Business House for Sale near Borey Chhork Vira...


In [ ]:
# df.loc[df['id'] == 'rl-133884683', 'price/m2'] = 360
# df.loc[df['id'] == 'rl-133884683', 'price'] = df[df['id'] == 'rl-133884683']['price/m2'] * df[df['id'] == 'rl-133884683']['land_size']

# df.loc[df['id'] == 'rl-121013970', 'price/m2'] = 3500
# df.loc[df['id'] == 'rl-121013970', 'price'] = df[df['id'] == 'rl-121013970']['price/m2'] * df[df['id'] == 'rl-121013970']['land_size']

# df.loc[df['id'] == 'rl-121013838', 'price/m2'] = 4200
# df.loc[df['id'] == 'rl-121013838', 'price'] = df[df['id'] == 'rl-121013838']['price/m2'] * df[df['id'] == 'rl-121013838']['land_size']

# df.loc[df['id'] == 'rl-121591501', 'price/m2'] = 600
# df.loc[df['id'] == 'rl-121591501', 'price'] = df[df['id'] == 'rl-121591501']['price/m2'] * df[df['id'] == 'rl-121591501']['land_size']

# df.loc[df['id'] == 'rl-121789127', 'price/m2'] = 2500
# df.loc[df['id'] == 'rl-121789127', 'price'] = df[df['id'] == 'rl-121789127']['price/m2'] * df[df['id'] == 'rl-121789127']['land_size']

# df.loc[df['id'] == 'rl-121014022', 'price/m2'] = 1260
# df.loc[df['id'] == 'rl-121014022', 'price'] = df[df['id'] == 'rl-121014022']['price/m2'] * df[df['id'] == 'rl-121014022']['land_size']



In [25]:
dfna = df[df['price'].isna()].reset_index(drop=True)
dfna


,id,information,type,province,price,land_size,price/m2,lat,long,translate
0,rl-133884683,ដីលក់បន្ទាន់⚡️នៅផ្លូវជាតិលេខ21ក្បែរផ្លូវក្រវ៉ា...,LAND,Kandal,NaN,1843.0,NaN,11.360443,105.005444,Land Emergency Land ⚡️ N by National Road 21 N...
1,rl-121013970,ID: CKOSIJJG\r\n🏠ដី សម្រាប់លក់ នៅសែនសុខ!\r\nដី...,LAND,Phnom Penh,NaN,3000.0,NaN,11.579789,104.857097,ID: ckosijjg\r\n🏠 Land for sale at Sen Sok\r\n...
2,rl-121013838,ដីនិងផ្ទះលក់ នៅទួលគោក ស្ថិតក្នុងតំបន់មានសក្តាន...,LAND,Phnom Penh,NaN,600.0,NaN,11.571095,104.888157,Land and home in Toul Kork is potentially in t...
3,rl-121591501,ដីសម្រាប់លក់នៅខណ្ឌដង្កោរ\r\n* នៅជិតតំបន់សេដ្ឋក...,LAND,Phnom Penh,NaN,NaN,NaN,11.439065,104.854218,Land for sale in Khan Dangkor\r\n* Near the Sp...
4,rl-121789127,ដីសម្រាប់លក់នៅសង្កាត់ភ្នំពេញថ្មី\r\n* នៅជិតផ្ស...,LAND,Phnom Penh,NaN,600.0,NaN,11.573510,104.877057,Land for sale in Sangkat Phnom Penh Thmey\r\n*...
...,...,...,...,...,...,...,...,...,...,...
73,rl-135329062,ដីមានបន្ទប់ជួល លក់បន្ទាន់⚡️នៅក្រោយផ្សារបឹងបៃតង...,LAND,Phnom Penh,NaN,640.0,NaN,11.624212,104.857043,Land has an instant rental\r\n👉 Price: $ 950 /...
74,rl-135327262,ដីកែងខ្នាតវីឡាលក់បន្ទាន់⚡️នៅជិតផ្លូវជាតិលេខ21 ...,LAND,Kandal,NaN,1800.0,NaN,11.360443,105.005444,Porn Land Seal Emergency Sale ⚡️ Near National...
75,rl-133983503,ដីនិងផ្ទះលក់⚡️នៅជិតផ្លូវជាតិលេខ២ ក្រុងតាខ្មៅ (...,RESIDENTIAL,Kandal,NaN,1800.0,NaN,11.481382,104.944869,Land & Home Sales ⚡️ Near National Road No 2 T...
76,rl-132369484,8 Bedroom Villa for Sale Area Toul Kork\r\n- S...,RESIDENTIAL,Phnom Penh,NaN,717.0,NaN,11.576372,104.893521,8 BedRoom Villa for Sale Area Toul Kork\r\n- S...


In [26]:
df.to_csv('../../data/raw/century21_v2.csv', index=False)